In [1]:
import psycopg2
from psycopg2  import extras
from sqlalchemy import create_engine
from sqlalchemy.orm import scoped_session, sessionmaker
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey, Date

DATABASE = 'postgresql'
USER = 'root'
PASSWORD = 'root'
HOST = 'postgres' # shared-networkに所属しているpostgres containerを指定できる
PORT = '5432'
DB_NAME = 'sandbox'

CONNECT_STR = '{}://{}:{}@{}:{}/{}'.format(DATABASE, USER, PASSWORD, HOST, PORT, DB_NAME)
engine = create_engine(CONNECT_STR)

conn = engine.connect()
res = conn.execute("select datname, datdba, encoding, datcollate, datctype from pg_database;")

for row in res:
    print(row)

('postgres', 10, 6, 'ja_JP.UTF-8', 'ja_JP.UTF-8')
('sandbox', 10, 6, 'ja_JP.UTF-8', 'ja_JP.UTF-8')
('template1', 10, 6, 'ja_JP.UTF-8', 'ja_JP.UTF-8')
('template0', 10, 6, 'ja_JP.UTF-8', 'ja_JP.UTF-8')


In [2]:
# create table
tablename = 'x007_products'

q1 = "SELECT * FROM information_schema.tables WHERE table_name = '{}';".format(tablename)
res = conn.execute(q1)

q2 = """
CREATE TABLE  {} (
    name varchar(256) NOT NULL,
    price integer NOT NULL
);
""".format(tablename)

if sum(1 for _ in res) == 0:
    print("create")
    conn.execute(q2)
    res = conn.execute(q1)
    sum(1 for _ in res)

In [3]:
# insert data
data = (
    ('りんご', 100),
    ('みかん', 50),
    ('バナナ', 100),
)

conn.execute("DELETE FROM {};".format(tablename))

metadata = MetaData()
tbl = Table(tablename, metadata,
            Column('name', Date),
            Column('price', Integer),
           )

for i in data:
    ins = tbl.insert().values(name=i[0], price=i[1])
    conn.execute(ins)

res = conn.execute("SELECT * FROM {};".format(tablename))
for i in res:
    print(i)

('りんご', 100)
('みかん', 50)
('バナナ', 100)


In [4]:
# 重複順列を得るSQL

q = f"""
select p1.name as name1, p2.name as name2
    from {tablename} p1 cross join {tablename} p2;
"""

print(q)

res = conn.execute(q)
for i in res:
    print(i)


select p1.name as name1, p2.name as name2
    from x007_products p1 cross join x007_products p2;

('りんご', 'りんご')
('りんご', 'みかん')
('りんご', 'バナナ')
('みかん', 'りんご')
('みかん', 'みかん')
('みかん', 'バナナ')
('バナナ', 'りんご')
('バナナ', 'みかん')
('バナナ', 'バナナ')


In [5]:
# 順列を得るSQL

q = f"""
select p1.name as name1, p2.name as name2
    from {tablename} p1 inner join {tablename} p2
        on p1.name <> p2.name;
"""

print(q)

res = conn.execute(q)
for i in res:
    print(i)


select p1.name as name1, p2.name as name2
    from x007_products p1 inner join x007_products p2
        on p1.name <> p2.name;

('りんご', 'みかん')
('りんご', 'バナナ')
('みかん', 'りんご')
('みかん', 'バナナ')
('バナナ', 'りんご')
('バナナ', 'みかん')


In [6]:
# 値段が同じ商品の組み合わせを取得する

q = f"""
select distinct p1.name, p1.price
    from {tablename} p1 inner join {tablename} p2
        on p1.price = p2.price
        and p1.name <> p2.name
order by p1.price;
"""

print(q)

res = conn.execute(q)
for i in res:
    print(i)


select distinct p1.name, p1.price
    from x007_products p1 inner join x007_products p2
        on p1.price = p2.price
        and p1.name <> p2.name
order by p1.price;

('りんご', 100)
('バナナ', 100)


In [7]:
# 重複組み合わせ

q = f"""
select p1.name, p2.name
    from {tablename} p1 inner join {tablename} p2
    on p1.name >= p2.name
order by p1.name desc;
"""

print(q)

res = conn.execute(q)
for i in res:
    print(i)


select p1.name, p2.name
    from x007_products p1 inner join x007_products p2
    on p1.name >= p2.name
order by p1.name desc;

('バナナ', 'りんご')
('バナナ', 'みかん')
('バナナ', 'バナナ')
('りんご', 'りんご')
('りんご', 'みかん')
('みかん', 'みかん')


In [8]:
# insert data
data = (
    ('りんご', 50),
    ('みかん', 100),
    ('ぶどう', 50),
    ('スイカ', 80),
    ('レモン', 30),
    ('バナナ', 50)
)

conn.execute("DELETE FROM {};".format(tablename))

metadata = MetaData()
tbl = Table(tablename, metadata,
            Column('name', Date),
            Column('price', Integer),
           )

for i in data:
    ins = tbl.insert().values(name=i[0], price=i[1])
    conn.execute(ins)

res = conn.execute("SELECT * FROM {};".format(tablename))
for i in res:
    print(i)

('りんご', 50)
('みかん', 100)
('ぶどう', 50)
('スイカ', 80)
('レモン', 30)
('バナナ', 50)


In [9]:
# 値段ランキング

q = f"""
select name, price,
    rank() over (order by price desc) as rank1,
    dense_rank() over (order by price desc) as rank2
from {tablename};
"""

print(q)

res = conn.execute(q)
for i in res:
    print(i)


select name, price,
    rank() over (order by price desc) as rank1,
    dense_rank() over (order by price desc) as rank2
from x007_products;

('みかん', 100, 1, 1)
('スイカ', 80, 2, 2)
('りんご', 50, 3, 3)
('ぶどう', 50, 3, 3)
('バナナ', 50, 3, 3)
('レモン', 30, 6, 4)


In [10]:
# 値段ランキング

q = f"""
select p1.name, p1.price,
    (select count(p2.price)
        from {tablename} p2
        where p2.price > p1.price) + 1 as rank1,
    (select count(distinct p2.price)
        from {tablename} p2
        where p2.price > p1.price) + 1 as rank2
from {tablename} p1
order by rank1 asc;
"""

print(q)

res = conn.execute(q)
for i in res:
    print(i)


select p1.name, p1.price,
    (select count(p2.price)
        from x007_products p2
        where p2.price > p1.price) + 1 as rank1,
    (select count(distinct p2.price)
        from x007_products p2
        where p2.price > p1.price) + 1 as rank2
from x007_products p1
order by rank1 asc;

('みかん', 100, 1, 1)
('スイカ', 80, 2, 2)
('りんご', 50, 3, 3)
('ぶどう', 50, 3, 3)
('バナナ', 50, 3, 3)
('レモン', 30, 6, 4)


In [11]:
# insert data
data = (
    ('りんご', 50),
    ('みかん', 100),
    ('みかん', 200),
    ('みかん', 100),
    ('バナナ', 50)
)

conn.execute("DELETE FROM {};".format(tablename))

metadata = MetaData()
tbl = Table(tablename, metadata,
            Column('name', Date),
            Column('price', Integer),
           )

for i in data:
    ins = tbl.insert().values(name=i[0], price=i[1])
    conn.execute(ins)

res = conn.execute("SELECT * FROM {};".format(tablename))
for i in res:
    print(i)

('りんご', 50)
('みかん', 100)
('みかん', 200)
('みかん', 100)
('バナナ', 50)


In [12]:
# 重複行の抽出

q = f"""
select * from
(select row_number() over (
    partition by name, price
        order by name) as row_num,
    name, price
from {tablename}) p
where p.row_num > 1;
"""

print(q)

res = conn.execute(q)
for i in res:
    print(i)


select * from
(select row_number() over (
    partition by name, price
        order by name) as row_num,
    name, price
from x007_products) p
where p.row_num > 1;

(2, 'みかん', 100)
